In [1]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from dotenv import load_dotenv
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
import os

In [2]:
load_dotenv()
# Load HUGGINGFACE_API_KEY from environment 
api_key = os.environ.get("HUGGINGFACEHUB_API_TOKEN")
if not api_key:
    raise RuntimeError('HUGGINGFACEHUB_API_TOKEN not set. Add it to .env or set the environment variable.')

In [3]:
#  Initialize Endpoint with the correct task
llm = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-R1",
    task="text-generation",  
    
)

# 3. Wrap in Chat Interface
model = ChatHuggingFace(llm=llm)

f:\AI Reasearcher\langgraph-tutorial\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
#Define the state
class BlogState(TypedDict):
    title: str
    outline: str
    content: str

In [12]:
def create_outline(state: BlogState) -> BlogState:
    # Fetch the title 
    title=state['title']
    # Define the prompt template
    prompt = (f"Generate a detailed outline for a blog post for this title: {title}")
    # Get the answer from the model
    outline=model.invoke(prompt).content

    # Update the state with the answer
    state['outline']=outline
    return state

In [13]:
def write_content(state: BlogState) -> BlogState:
    # Fetch the title 
    title=state['title']
    outline=state['outline']
    # Define the prompt template
    prompt = (f"Generate a detailed blog post for this title: {title} and outline\n {outline}")
    # Get the answer from the model
    content=model.invoke(prompt).content
    # Update the state with the answer
    state['content']=content
    return state

In [14]:
# define the graph
graph=StateGraph(BlogState)

# add nodes
graph.add_node('create_outline',create_outline)
graph.add_node('write_content',write_content)

# add edges
graph.add_edge(START,'create_outline')
graph.add_edge('create_outline','write_content')
graph.add_edge('write_content',END)
# compile the graph
workflow=graph.compile()


In [15]:
initial_state={'title':'The Future of AI'}
final_state=workflow.invoke(initial_state)
print(final_state['content'])

<think>
We are generating a detailed outline for a blog post titled "The Future of AI." The user hasn't specified a particular audience or length, so we'll aim for a comprehensive yet accessible structure that covers key aspects of AI's evolution. Let's break this down thoughtfully.

First, consider the user's likely scenario: They could be a content creator, tech professional, or educator needing a structured resource. Their request suggests they want more than surface-level predictions—they need actionable insights and balanced perspectives. The unspoken need here is probably clarity amid AI's overwhelming hype cycle. They might fear missing critical implications or want to prepare readers for real-world impacts.

For the outline's framework, I'll start with an engaging hook to address AI's accelerated pace, then ground readers in current capabilities before projecting forward. This builds credibility. The meat should explore both transformative potentials and sobering risks—this dua

In [16]:
print(final_state['outline'])

<think>
Okay, the user asked me to generate a detailed outline for a blog post titled "The Future of AI." That sounds straightforward, but I should think about who might be reading this blog. Probably tech enthusiasts, professionals, or curious readers interested in AI's impact. They'd want something comprehensive yet engaging, not just a dry list.  

Hmm, the user didn't specify a target audience or length, so I'll aim for a balanced approach—detailed but not overwhelming. I should cover key areas like current trends, ethical concerns, future predictions, and practical advice.  

First, I need a hook to grab attention. Maybe start with a vivid metaphor about AI evolving fast—like comparing it to a rocket launch—to emphasize urgency. Then, I'll structure it logically: introduction, current state, trends, challenges, future scenarios, and actionable takeaways.  

For the current state section, I should mention things like LLMs and generative AI because they're hot topics. But I must kee